##구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
IMAGE_width=72
IMAGE_height=72
IMAGE_size=(IMAGE_width,IMAGE_height)
IMAGE_channels=1

## 파일 경로 지정

In [ ]:
train_path='/gdrive/My Drive/생체인증보안/지문/fingerprint_train'
test_path='/gdrive/My Drive/생체인증보안/지문/fingerprint_test'

## 라이브러리 import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
from keras.preprocessing import image
from keras.models import Model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import tensorflow as tf
import random
import glob, os
import cv2
from PIL import Image

## Augmentation 후 저장

In [ ]:
seq = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0, 0.5)),
    iaa.Affine(
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        rotate=(-30, 30),
        order=[0, 1],
        cval=255
    )
], random_order=True)

In [ ]:
#1~8까지 반복
%cd '/gdrive/My Drive/생체인증보안/지문/fingerprint_train/1'
nowpath='/gdrive/My Drive/생체인증보안/지문/fingerprint_train/1'

for filename in os.listdir(nowpath):
  img=Image.open(filename)
  img=img.convert('L')
  img=np.array(img)
  n=80
  augs=[img]*n
  augs=seq.augment_images(augs)

  for i in range(n):
    img=Image.fromarray(augs[i])
    fname=filename+'_'+str(i)
    img.save(fname,'bmp')

##데이터 불러오기

In [ ]:
x_data=[]
y_data=[]
folder_names=['1','2','3','4','5','6','7','8']

for folder_name in folder_names:
  file_names = os.listdir('{}/{}'.format(train_path, folder_name))
  for file_name in file_names:
    file_path='{}/{}/{}'.format(train_path, folder_name, file_name)
    image=Image.open(file_path)
    image=image.resize((IMAGE_width,IMAGE_height))
    np_image=np.asarray(image)
    np_image=np_image/256

    x_data.append(np_image)
    y_data.append(int(folder_name)-1)


In [ ]:
x_data = np.array(x_data)
y_data = np.array(y_data)

In [ ]:
x_data=x_data.reshape(6480,72,72,1)
x_data.shape

(6480, 72, 72, 1)

##모델링

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(5,5),strides=(1,1),
                         padding='same', activation='relu', 
                         input_shape=(IMAGE_width, IMAGE_height, IMAGE_channels)),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),

  tf.keras.layers.Conv2D(64, (3,3), activation='relu',padding='same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
  tf.keras.layers.Dropout(0.25),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1000, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(8, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 72, 72, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 36, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 18, 18, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20736)             0         
_________________________________________________________________
dense (Dense)                (None, 1000)              2

##Callback

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
#조기종료
earlystop=EarlyStopping(monitor='sparse_categorical_accuracy',patience=10)

#학습율 조정
learning_rate_reduction=ReduceLROnPlateau(monitor='sparse_categorical_accuracy',
                                          patience=2,
                                          verbose=1,
                                          factor=0.5,
                                          min_lr=0.00001)
#callback 설정
callback=[earlystop,learning_rate_reduction]

##모델 학습(Cross-Validation)

In [ ]:
from sklearn.model_selection import StratifiedKFold

kfold=StratifiedKFold(n_splits=3)
cvscores = []

for train_idx,val_idx in kfold.split(x_data,y_data):
  x_train,x_val=x_data[train_idx],x_data[val_idx]
  y_train,y_val=y_data[train_idx],y_data[val_idx]

  model.fit(x_train,y_train,epochs=30,batch_size=20,verbose=1,callbacks=callback)
  
  scores = model.evaluate(x_val, y_val, verbose=1)
  print("Acc: ",(scores[1]*100))
  cvscores.append(scores[1] * 100)

Epoch 1/30
216/216 [==============================] - 71s 326ms/step - loss: 5.8532e-04 - sparse_categorical_accuracy: 1.0000
Epoch 2/30
216/216 [==============================] - 70s 324ms/step - loss: 0.0015 - sparse_categorical_accuracy: 0.9998
Epoch 3/30
216/216 [==============================] - 70s 323ms/step - loss: 0.0011 - sparse_categorical_accuracy: 1.0000
Epoch 4/30
216/216 [==============================] - 70s 325ms/step - loss: 8.6864e-04 - sparse_categorical_accuracy: 0.9998
Epoch 5/30
216/216 [==============================] - 71s 327ms/step - loss: 8.5485e-04 - sparse_categorical_accuracy: 1.0000
Epoch 6/30
216/216 [==============================] - 71s 328ms/step - loss: 0.0010 - sparse_categorical_accuracy: 1.0000
Epoch 7/30
216/216 [==============================] - 71s 331ms/step - loss: 0.0012 - sparse_categorical_accuracy: 0.9998
Epoch 8/30
216/216 [==============================] - 71s 331ms/step - loss: 0.0010 - sparse_categorical_accuracy: 0.9998
Epoch 9/30
2

In [ ]:
print("평균 Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

평균 Accuracy: 100.00% (+/- 0.00%)


##모델 성능 지표</br>
학습 데이터 중 split한 validation 데이터 사용

In [ ]:
y_val_pred=model.predict(x_val)
y_val_pred

array([[1.0000000e+00, 6.6024852e-10, 1.2501187e-18, ..., 3.0993702e-15,
        1.2012045e-19, 3.1174573e-20],
       [1.0000000e+00, 7.4570777e-10, 2.3471680e-17, ..., 1.6066247e-13,
        1.1927265e-17, 4.0163758e-17],
       [9.9999988e-01, 8.4333998e-08, 1.6313076e-11, ..., 7.5400247e-15,
        6.3779987e-10, 6.2816376e-16],
       ...,
       [2.0817796e-15, 7.0318287e-12, 4.2005957e-23, ..., 5.7909000e-13,
        3.0747454e-08, 1.0000000e+00],
       [9.7130831e-11, 1.3791428e-12, 1.3908549e-21, ..., 9.0744598e-14,
        7.1945756e-08, 9.9999988e-01],
       [9.8605935e-10, 2.1622205e-07, 2.1062787e-15, ..., 9.5671266e-12,
        2.0567038e-04, 9.9979407e-01]], dtype=float32)

In [ ]:
print(y_val)

[0 0 0 ... 7 7 7]


In [ ]:
y_pred_val=[]
for i in range(len(y_val)):
  n=np.argmax(y_val_pred[i])
  y_pred_val.append(n)

In [ ]:
print(y_pred_val)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(y_val,y_pred_val)                         
precision = precision_score(y_val,y_pred_val,average='micro') 
recall = recall_score(y_val,y_pred_val, average='micro')      
f1 = f1_score(y_val,y_pred_val, average='micro')

print('정확도: {0:.6f}, 정밀도: {1:.6f}, 재현율: {2:.6f}, F1-score: {3:.6f}'.
      format(accuracy, precision, recall, f1))

정확도: 1.000000, 정밀도: 1.000000, 재현율: 1.000000, F1-score: 1.000000


##테스트

In [ ]:
%cd '/gdrive/My Drive/생체인증보안/지문/fingerprint_test'

x_test=[]
y_test=[]
for filename in os.listdir(test_path):
  img=Image.open(filename)
  print(filename)
  img=img.resize((IMAGE_width,IMAGE_height))
  np_image=np.asarray(img)
  np_image=np_image/256

  x_test.append(np_image)

x_test = np.array(x_test)

/gdrive/My Drive/생체인증보안/지문/fingerprint_test
1.bmp
41.bmp
79.bmp
64.bmp
69.bmp
19.bmp
43.bmp
26.bmp
67.bmp
36.bmp
25.bmp
31.bmp
17.bmp
24.bmp
27.bmp
60.bmp
45.bmp
77.bmp
29.bmp
50.bmp
7.bmp
59.bmp
9.bmp
18.bmp
49.bmp
30.bmp
58.bmp
28.bmp
13.bmp
33.bmp
16.bmp
74.bmp
4.bmp
11.bmp
21.bmp
40.bmp
8.bmp
65.bmp
44.bmp
71.bmp
5.bmp
76.bmp
78.bmp
15.bmp
2.bmp
22.bmp
61.bmp
62.bmp
32.bmp
37.bmp
75.bmp
46.bmp
57.bmp
3.bmp
34.bmp
23.bmp
14.bmp
12.bmp
6.bmp
56.bmp
55.bmp
10.bmp
72.bmp
48.bmp
20.bmp
68.bmp
47.bmp
66.bmp
38.bmp
73.bmp
42.bmp
52.bmp
54.bmp
51.bmp
80.bmp
39.bmp
53.bmp
70.bmp
63.bmp
35.bmp


In [ ]:
x_test=x_test.reshape(80,72,72,1)
x_test.shape

(80, 72, 72, 1)

In [ ]:
y_pred=model.predict(x_test)
print(y_pred)

In [ ]:
for i in range(80):
  n=np.argmax(y_pred[i])+1
  print(n)

7
7
7
7
7
8
8
7
7
7
8
8
8
8
8
8
8
8
8
8
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
